In [ ]:
# =========================
# 설치 (Colab 전용)
# =========================
!pip install -q openai gradio

In [6]:
from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# =========================
# 라이브러리
# =========================
import os
import gradio as gr
from openai import OpenAI
import evaluate

# =========================
# 번역 함수 (스트리밍 지원)
# =========================
SYSTEM_PROMPT_TEMPLATE = """너는 전문 번역가다. 반드시 {target_lang}로만 번역해라.
[지침]
- 원문 의미를 정확히 보존하되, {domain_style}
- 과도한 의역/설명 금지, 번역문만 출력
- 줄바꿈/리스트/마크다운은 가능한 유지
"""

DOMAIN_STYLES = {
    "일상": "자연스럽고 읽기 쉬운 구어체로 표현한다.",
    "IT": "기술 용어/약어/표기를 보존하고, 정확한 기술 문서 톤으로 표현한다."
}

def translate(text: str, direction: str, domain: str, api_key: str):
    if not text.strip():
        yield " 번역할 텍스트를 입력하세요."
        return

    key = api_key or os.getenv("OPENAI_API_KEY")
    if not key:
        yield " OpenAI API 키가 없습니다. 입력칸에 키를 넣거나 환경변수 OPENAI_API_KEY를 설정하세요."
        return

    client = OpenAI(api_key=key)

    target_lang = "영어" if direction == "한→영" else "한국어"
    src_lang = "한국어" if direction == "한→영" else "영어"

    sys_prompt = SYSTEM_PROMPT_TEMPLATE.format(
        target_lang=target_lang,
        domain_style=DOMAIN_STYLES.get(domain, "문맥에 맞게 자연스럽게 번역한다.")
    )

    messages = [
        {"role": "system", "content": sys_prompt},
        {
            "role": "user",
            "content": (
                f"원문({src_lang}):\n{text}\n\n"
                f"번역 대상 언어: {target_lang}\n"
                f"번역문만 출력해줘."
            ),
        },
    ]

    try:
        stream = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.2,
            stream=True,
        )
        out = ""
        for chunk in stream:
            try:
                delta = chunk.choices[0].delta
                if delta and getattr(delta, "content", None):
                    out += delta.content
                    yield out  # Gradio에 실시간 갱신
            except Exception:
                pass
    except Exception as e:
        yield f" 오류: {e}"

def evaluateByGPT(origin_str, trans_str, ref_str, direction, api_key):

    lang = "en" if direction == "한→영" else "kr"
    # 4. GPT 평가 프롬프트 (한글 버전)
    prompt = f"""
    당신은 평가자입니다. 다음 두 번역을 비교해 주세요.

    📌 원문:
    {origin_str}

    📌 참조 번역:
    {ref_str}

    📌 모델 번역:
    {trans_str}

    평가 기준 (0~5점):
    - 5점 = 완벽하게 일치
    - 4점 = 거의 일치하나 사소한 차이 있음
    - 3점 = 부분적으로 맞지만 중요한 차이가 있음
    - 2점 = 대부분 틀렸지만 일부 관련성 있음
    - 1점 = 거의 틀림
    - 0점 = 전혀 관련 없음

    출력 형식:
    점수: X/5
    이유: 간단한 설명
    """

    key = api_key or os.getenv("OPENAI_API_KEY")
    if not key:
        yield " OpenAI API 키가 없습니다. 입력칸에 키를 넣거나 환경변수 OPENAI_API_KEY를 설정하세요."
        return

    client = OpenAI(api_key=key)

    # 5. GPT API 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    print("🔍 GPT 평가 결과:")
    print(response.choices[0].message.content)
    out = response.choices[0].message.content

    bertscore = evaluate.load("bertscore")

    # 9. BERTScore
    bertscore_result = bertscore.compute(predictions=[trans_str], references=[ref_str], lang="en")
    print("\nBERTScore (F1):", bertscore_result["f1"][0])

    out += "\n" + f'\nBERTScore (F1):{bertscore_result["f1"][0]:.4f}'
    yield out  # Gradio에 실시간 갱신


# =========================
# Gradio UI
# =========================
with gr.Blocks(title="GPT 한↔영 번역기") as demo:
    gr.Markdown("#  GPT 한↔영 번역기\n- GPT API와 Gradio로 구현된 간단 번역기\n- 번역 방향과 도메인(일상/IT)을 선택하세요.")

    with gr.Row():
        api_key = gr.Textbox(
            label="OpenAI API Key (선택)",
            type="password",
            placeholder="환경변수 OPENAI_API_KEY 설정 시 생략 가능"
        )
        direction = gr.Radio(
            ["한→영", "영→한"],
            value="한→영",
            label="번역 방향"
        )
        domain = gr.Dropdown(
            ["일상", "IT"],
            value="일상",
            label="도메인 스타일"
        )

    src = gr.Textbox(lines=3, label="입력 텍스트")    
    tgt = gr.Textbox(lines=3, label="번역 결과", interactive=False)    

    with gr.Row():
        btn = gr.Button("번역", variant="primary")
        clr = gr.Button("초기화")        

    ref_src = gr.Textbox(lines=2, label="참조 번역 텍스트")
    eval = gr.Textbox(lines=8, label="평가 결과", interactive=True)
    eva = gr.Button("평가")

    gr.Examples(
        examples=[
            ["오늘 회의 자료를 공유해 주세요.", "한→영", "일상", "", "Please share today's meeting materials."],
            ["We are deploying the new API gateway this Friday.", "영→한", "IT", "", "이번 금요일에 새로운 API 게이트웨이를 배포할 예정입니다."],
        ],
        inputs=[src, direction, domain, api_key, ref_src],
        label="예시"
    )

    # 클릭 이벤트: 스트리밍(제너레이터) 연결
    btn.click(
        fn=translate,
        inputs=[src, direction, domain, api_key],
        outputs=tgt
    )

    # 클릭 이벤트: 스트리밍(제너레이터) 연결 평가
    eva.click(
        fn=evaluateByGPT,
        inputs=[src, tgt, ref_src, direction, api_key],
        outputs=eval
    )

    # 초기화 버튼
    clr.click(lambda: ("", ""), None, [src, tgt])

demo.launch()

In [ ]:
demo.close()

Closing server running on port: 7864


: 